In [1]:
class C3Dmas(nn.Module):
    """
    The C3D network as described in [1].
    """

    def __init__(self):
        super(C3Dmas, self).__init__()

        self.conv1 = nn.Conv3d(3, 64, kernel_size=(3, 3, 3), padding=(1, 1, 1))
        self.pool1 = nn.MaxPool3d(kernel_size=(1, 2, 2), stride=(1, 2, 2), return_indices=True)

        self.conv2 = nn.Conv3d(64, 128, kernel_size=(3, 3, 3), padding=(1, 1, 1))
        self.pool2 = nn.MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2), return_indices=True)

        self.conv3a = nn.Conv3d(128, 256, kernel_size=(3, 3, 3), padding=(1, 1, 1))
        self.conv3b = nn.Conv3d(256, 256, kernel_size=(3, 3, 3), padding=(1, 1, 1))
        self.pool3 = nn.MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2), return_indices=True)

        self.conv4a = nn.Conv3d(256, 512, kernel_size=(3, 3, 3), padding=(1, 1, 1))
        self.conv4b = nn.Conv3d(512, 512, kernel_size=(3, 3, 3), padding=(1, 1, 1))
        self.pool4 = nn.MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2), return_indices=True)

        self.conv5a = nn.Conv3d(512, 512, kernel_size=(3, 3, 3), padding=(1, 1, 1))
        self.conv5b = nn.Conv3d(512, 512, kernel_size=(3, 3, 3), padding=(1, 1, 1))
        self.pool5 = nn.MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2), padding=(0, 1, 1))

        self.fc6 = nn.Linear(8192, 4096)
        self.fc7 = nn.Linear(4096, 4096)
        self.fc8 = nn.Linear(4096, 2)  # 487 class in sport1m

        self.dropout = nn.Dropout(p=0.5)

        self.relu = nn.ReLU()
        self.softmax = nn.LogSoftmax()

    def forward(self, x):
        h = self.relu(self.conv1(x))
        h = self.pool1(h)

        h = self.relu(self.conv2(h))
        h = self.pool2(h)

        h = self.relu(self.conv3a(h))
        h = self.relu(self.conv3b(h))
        h = self.pool3(h)

        h = self.relu(self.conv4a(h))
        h = self.relu(self.conv4b(h))
        h = self.pool4(h)

        h = self.relu(self.conv5a(h))
        h = self.relu(self.conv5b(h))
        h = self.pool5(h)

        h = h.view(-1, 8192)
        h = self.relu(self.fc6(h))
        h = self.dropout(h)
        h = self.relu(self.fc7(h))
        h = self.dropout(h)

        logits = self.fc8(h)
        probs = self.softmax(logits)

        return probs



NameError: name 'nn' is not defined

In [ ]:

    # load net with color input
    colorNet = C3Dmas()
    colorNet.load_state_dict(torch.load("weight.t7"))
    colorNet.eval()

    convForward = C3Dmas()
    # convForward = convForward.cuda()

    convForward.conv1 = colorNet.conv1
    convForward.conv2 = colorNet.conv2
    convForward.conv3a = colorNet.conv3a
    convForward.conv3b = colorNet.conv3b
    convForward.conv4a = colorNet.conv4a
    convForward.conv4b = colorNet.conv4b
    convForward.conv5a = colorNet.conv5a
    convForward.conv5b = colorNet.conv5b

    # forward path in Network

    q = (convForward.conv1(vid))
    # conv1 = q
    q = F.relu(q)
    pool1output, indicesPool1 = convForward.pool1(q)
    q = pool1output

    q = F.relu(convForward.conv2(q))
    pool2output, indicesPool2 = convForward.pool2(q)
    q = pool2output

    q = F.relu(convForward.conv3a(q))
    q = F.relu(convForward.conv3b(q))
    pool3output, indicesPool3 = convForward.pool3(q)
    q = pool3output

    q = F.relu(convForward.conv4a(q))
    q = F.relu(convForward.conv4b(q))
    pool4output, indicesPool4 = convForward.pool4(q)
    q = pool4output

    q = F.relu(convForward.conv5a(q))
    q = F.relu(convForward.conv5b(q))

    # print(q)



    deconv5b = nn.ConvTranspose3d(512, 512, kernel_size=(3, 3, 3), padding=(1, 1, 1))
    deconv5a = nn.ConvTranspose3d(512, 512, kernel_size=(3, 3, 3), padding=(1, 1, 1))

    unpool4 = nn.MaxUnpool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2))
    deconv4b = nn.ConvTranspose3d(512, 512, kernel_size=(3, 3, 3), padding=(1, 1, 1))
    deconv4a = nn.ConvTranspose3d(512, 256, kernel_size=(3, 3, 3), padding=(1, 1, 1))

    unpool3 = nn.MaxUnpool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2))
    deconv3b = nn.ConvTranspose3d(256, 256, kernel_size=(3, 3, 3), padding=(1, 1, 1))
    deconv3a = nn.ConvTranspose3d(256, 128, kernel_size=(3, 3, 3), padding=(1, 1, 1))

    unpool2 = nn.MaxUnpool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2))
    deconv2 = nn.ConvTranspose3d(128, 64, kernel_size=(3, 3, 3), padding=(1, 1, 1))

    unpool1 = nn.MaxUnpool3d(kernel_size=(1, 2, 2), stride=(1, 2, 2))
    deconv1 = nn.ConvTranspose3d(64, 3, kernel_size=(3, 3, 3), padding=(1, 1, 1))

    h = F.relu(deconv5b(F.relu(q)))
    h = (deconv5a(h))
    h = F.relu(unpool4(h, indicesPool4))
    h = F.relu(deconv4b(h))
    h = F.relu(deconv4a(h))
    h = F.relu(unpool3(h, indicesPool3))
    h = F.relu(deconv3b(h))
    h = (deconv3a(h))
    h = F.relu(unpool2(h, indicesPool2))
    h = ((deconv2(h)))
    h = F.relu(unpool1(h, indicesPool1))
    h = (deconv1(h))